In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error,r2_score
import plotly.express as px
import plotly.graph_objects as go
from lstm import SequentialLSTM
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import joblib


In [2]:
model = SequentialLSTM(input_size=1, output_size=1)  # Ensure input and output sizes match your saved model
model.load_state_dict(torch.load("./models/lstm.pth"))
model.eval()
model.to(device)
scaler = joblib.load('scaler.pkl')
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed = 69)

/tmp/ipykernel_46193/2353787790.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./models/lstm.pth"))


In [3]:
df = pd.read_csv('./test.csv')
df = df.sort_values(by="Date")
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-04-21,508.000000,515.460022,503.600006,508.899994,508.899994,22897400
1,2021-04-22,513.820007,513.960022,500.549988,508.779999,508.779999,9061100
2,2021-04-23,509.010010,509.700012,500.700012,505.549988,505.549988,7307700
3,2021-04-26,506.760010,510.480011,503.000000,510.299988,510.299988,4388800
4,2021-04-27,512.619995,512.989990,504.579987,505.549988,505.549988,3761300


In [4]:
dataset = pd.DataFrame(df.reset_index()["Close"])
data = dataset.values.astype('float32')


In [5]:
data = scaler.transform(data)
data

array([[0.6007296 ],
       [0.60046744],
       [0.59341204],
       [0.60378754],
       [0.59341204],
       [0.59553087],
       [0.600948  ],
       [0.6107118 ],
       [0.6011882 ],
       [0.58823526],
       [0.5727266 ],
       [0.5803062 ],
       [0.58967686],
       [0.55221593],
       [0.57054234],
       [0.54848087],
       [0.5521505 ],
       [0.56680715],
       [0.5571307 ],
       [0.55132043],
       [0.55442214],
       [0.584937  ],
       [0.5766803 ],
       [0.5876236 ],
       [0.5842161 ],
       [0.58644414],
       [0.5897206 ],
       [0.587427  ],
       [0.57927954],
       [0.57962906],
       [0.55820096],
       [0.56979966],
       [0.5696249 ],
       [0.5646666 ],
       [0.5502938 ],
       [0.5534829 ],
       [0.55675936],
       [0.58104897],
       [0.56359625],
       [0.56471026],
       [0.5776632 ],
       [0.5829711 ],
       [0.57473624],
       [0.6005548 ],
       [0.60911727],
       [0.6207378 ],
       [0.6404185 ],
       [0.653

In [6]:
def create_dataset(dataset, look_back=1):
 dataX, dataY = [], []
 for i in range(len(dataset)-look_back-1):
     a = dataset[i:(i+look_back), 0]
     dataX.append(a)
     dataY.append(dataset[i + look_back, 0])
 return np.array(dataX), np.array(dataY)

In [7]:
X, _ = create_dataset(data, look_back=1)
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

X = torch.tensor(X, dtype=torch.float32).to(device)

In [8]:
X

tensor([[[0.6007]],

        [[0.6005]],

        [[0.5934]],

        [[0.6038]],

        [[0.5934]],

        [[0.5955]],

        [[0.6009]],

        [[0.6107]],

        [[0.6012]],

        [[0.5882]],

        [[0.5727]],

        [[0.5803]],

        [[0.5897]],

        [[0.5522]],

        [[0.5705]],

        [[0.5485]],

        [[0.5522]],

        [[0.5668]],

        [[0.5571]],

        [[0.5513]],

        [[0.5544]],

        [[0.5849]],

        [[0.5767]],

        [[0.5876]],

        [[0.5842]],

        [[0.5864]],

        [[0.5897]],

        [[0.5874]],

        [[0.5793]],

        [[0.5796]],

        [[0.5582]],

        [[0.5698]],

        [[0.5696]],

        [[0.5647]],

        [[0.5503]],

        [[0.5535]],

        [[0.5568]],

        [[0.5810]],

        [[0.5636]],

        [[0.5647]],

        [[0.5777]],

        [[0.5830]],

        [[0.5747]],

        [[0.6006]],

        [[0.6091]],

        [[0.6207]],

        [[0.6404]],

        [[0.6

In [9]:
# add seed to ensure reproducibility

newPredictions = model(X).detach().cpu().numpy()
# Inverse transform the predictions
newPredictions = scaler.inverse_transform(newPredictions)
newPredictions


array([[510.87466],
       [510.72986],
       [507.11072],
       [512.4411 ],
       [507.11072],
       [508.19983],
       [510.98447],
       [515.9992 ],
       [511.10895],
       [504.44012],
       [496.47086],
       [500.3748 ],
       [505.18683],
       [485.93008],
       [495.33087],
       [484.0136 ],
       [485.8993 ],
       [493.41647],
       [488.4447 ],
       [485.46106],
       [487.05936],
       [502.74698],
       [498.49744],
       [504.1285 ],
       [502.37457],
       [503.51672],
       [505.20953],
       [504.0258 ],
       [499.83496],
       [500.01834],
       [489.00223],
       [494.9517 ],
       [494.8646 ],
       [492.3285 ],
       [484.9486 ],
       [486.5769 ],
       [488.25226],
       [500.7553 ],
       [491.76776],
       [492.35358],
       [499.0123 ],
       [501.7373 ],
       [497.49728],
       [510.78177],
       [515.1764 ],
       [521.14667],
       [531.21277],
       [537.83124],
       [538.34827],
       [532.4923 ],


In [10]:
look_back = 1
dates = df['Date'][look_back:len(newPredictions) + look_back].reset_index(drop=True)

# Create a DataFrame
results_df = pd.DataFrame({
    'Date': dates,
    'Actual': df['Close'][look_back:len(newPredictions) + look_back].reset_index(drop=True),
    'Predicted': newPredictions.flatten()
})

# Display the DataFrame
print(results_df.head(50))


          Date      Actual   Predicted
0   2021-04-22  508.779999  510.874664
1   2021-04-23  505.549988  510.729858
2   2021-04-26  510.299988  507.110718
3   2021-04-27  505.549988  512.441101
4   2021-04-28  506.519989  507.110718
5   2021-04-29  509.000000  508.199829
6   2021-04-30  513.469971  510.984467
7   2021-05-03  509.109985  515.999207
8   2021-05-04  503.179993  511.108948
9   2021-05-05  496.079987  504.440125
10  2021-05-06  499.549988  496.470856
11  2021-05-07  503.839996  500.374786
12  2021-05-10  486.690002  505.186829
13  2021-05-11  495.079987  485.930084
14  2021-05-12  484.980011  495.330872
15  2021-05-13  486.660004  484.013611
16  2021-05-14  493.369995  485.899292
17  2021-05-17  488.940002  493.416473
18  2021-05-18  486.279999  488.444702
19  2021-05-19  487.700012  485.461060
20  2021-05-20  501.670013  487.059357
21  2021-05-21  497.890015  502.746979
22  2021-05-24  502.899994  498.497437
23  2021-05-25  501.339996  504.128510
24  2021-05-26  502.35998